# Marta Balairón García: 100451724


# NETWORK OPTIMIZATION PROBLEM

## A TRIP AROUND EUROPE

We are organizing a trip around Europe. We  are going to start thre trip at Madrid and we intend to meet the following cities:

We want to obtain the most efficient route in terms of expending the less possible money. But we have to have in mind that each city is part of a "stage of the trip" and we cannot pass to the next stage before having visited all the cities of the previous stage.


### PROBLEM STATEMENT

We are going to treat **the cities as nodes**:
* Origin:
    - Madrid (1)
* 1st stage:
    - Paris (2)
    - Luxemburg (3)
    - Milan (4)
* 2nd stage:
    - Berlin (5)
    - Munich (6)
    - Viena (7)
* 3rd Stage:
    - Budapest (8)
    - Split (9)

    
The problem can be shown as a **network problem** where each of the cities represents a node and the arcs represent the flight between those cities with an associated cost. Moreover, the problem is stated as a **binary discrete problem** in which each of the variables (which represent a flight between two cities) takes the value 1 if the flight betweent the cities is done and 0 otherwise.

In the following picture we can observe the network with some of the connections. The graph is a complete directed graph, but in this picture is simplified in terms of making it more visual:
<img src="img\enunciado.png">


**The objective** of the problem is to **minimize the costs** of The European trip.

What is more, the model must follow some **constraints**:
- We must start visiting the cities of the stage 1 (2, 3, 4)
- We cannot change from stage 1 (2, 3, 4) to stage 2 (5, 6, 7) without having completed the stage 1
- We cannot change from stage 2 (5, 6, 7) to stage 3 (8, 9) without having completed the stage
- We must finish the trip in a stage 3 city



### PROBLEM FORMULATION

#### Sets 
- $C$ = set of cities (nodes)
- $origin$ = origin city (nodes)
- $stage1$ = set of cities of stage 1 (nodes)
- $stage2$ = set of cities of stage 2 (nodes)
- $stage3$ = set of cities of stage 3 (nodes)

#### Variables
$$
x_{ij}=
\left\{\begin{array}{ll} 
1, & \text{The node $i$ is visited just before node $j$,}\\
0, & \text{else}\\
\end{array} \right.\quad
$$

#### Parameters
$Cost$ = cost of a flight from city $i$ to city $j$ $\forall i, j \in C$ 

#### Objective
Minimize the costs of all flights
$$\min_{x_{ij}} \sum\nolimits_{i\neq j} c_{ij} x_{ij}$$

#### Constraints
- We must avoid cycles $x_{ij} + x_{ji} \leq1$
- There are not arrivals to origin city $\sum_{i\in C} x_{i1} = 0$ 
- We can arrive just once to the rest of the cities $\sum_{i\in C} x_{ij} = 0$, $\forall j \in C except 1$
- There is only one departure from each cities  $\sum_{j\in C} x_{ij} = 0$, $\forall i \in C except  8, 9$
- The flight must start in one stage 1 city  $\sum_{j\in Stage 1} x_{1j} = 1$
- We cannot flight from origin to a stage 2 or 3 city  $\sum_{j\in Stage 2  stage 3} x_{1j} = 0$
- There is only on flight form stage 1 to stage 2 $\sum_{i\in stage 1}\sum_{j \in stage 2} x_{ij} = 1$
- We cannot flight from stage 1 cities to a stage 3 city  $\sum_{i\in stage 2}\sum_{j \in stage 3} x_{ij} = 1$
- We must finish the trip in one of the stage 3 cities $\sum_{j\in C} x_{ij} \leq1 $, $\forall i\in Origin 3$



### CODING THE PROBLEM

#### Read and create a .dat file to store the data of the model

In [ ]:
%%writefile european_trip.dat
param n := 9;

param costs :=
    1 1 0
    1 2 100
    1 3 140 
    1 4 90
    1 5 200
    1 6 200
    1 7 200
    1 8 200
    1 9 200
    2 1 100
    2 2 0
    2 3 60
    2 4 80
    2 5 100 
    2 6 120
    2 7 120
    2 8 200
    2 9 200
    3 1 140
    3 2 60
    3 3  0
    3 4 120
    3 5 70
    3 6 50
    3 7 90
    3 8 200
    3 9 200
    4 1 90
    4 2 80
    4 3 110
    4 4 0
    4 5 130
    4 6 40
    4 7 80
    4 8 200
    4 9 200
    5 1 200
    5 2 100
    5 3 70
    5 4 130
    5 5 0
    5 6 80
    5 7 90
    5 8 80
    5 9 150
    6 1 200
    6 2 120
    6 3 40 
    6 4 50
    6 5 80
    6 6 0
    6 7 45
    6 8 60
    6 9 90
    7 1 200
    7 2 120
    7 3 90
    7 4 80
    7 5 90
    7 6 45
    7 7 0
    7 8 70
    7 9 70
    8 1 200
    8 2 200
    8 3 200
    8 4 200
    8 5 80
    8 6 60
    8 7 70
    8 8 0
    8 9 60
    9 1 200
    9 2 200
    9 3 200
    9 4 200
    9 5 150
    9 6 90
    9 7 70
    9 8 60
    9 9 0;


### code the problem

In [ ]:
%%writefile european_trip.py
from pyomo.environ import *
model = AbstractModel()

# initialize the nodes and the arcs
model.n = Param(within=NonNegativeIntegers)
model.I = Set(initialize=[1, 2, 3, 4, 5, 6, 7, 8, 9]) 
model.J = Set(initialize=[1, 2, 3, 4, 5, 6, 7, 8, 9])

model.origin= Set(initialize=[1]) 
model.stage1= Set(initialize=[2, 3, 4]) 
model.stage2= Set(initialize=[5, 6, 7]) 
model.stage3= Set(initialize=[8, 9]) 
model.stage2stage3 = Set(initialize=[5, 6, 7, 8 , 9])

# initialize the binary variables
model.x = Var(model.I, model.J, initialize=0, domain=Binary)

# initialize the costs
model.costs= Param(model.I, model.J)

# OBJECTIVE FUNCTION
# minimize the cost
def obj_rule(model):
    return sum(model.x[i,j]*model.costs[i, j] for i in model.I for j in model.J)

model.objective = Objective(rule = obj_rule, sense = minimize)
model.objective.pprint()

# CONSTRAINTS
def cycles(model, i, j):
    return model.x[i,j]+model.x[j,i] <= 1

def arrivals(model, j):
    if j != 1:
        return sum(model.x[i,j] for i in model.I if i!=j) == 1
    else:
        return sum(model.x[i,j] for i in model.I if i!=j) == 0

def departures(model, i):
    if (i != 8) and (i !=9):
        return sum(model.x[i,j] for j in model.J if i!=j) == 1
    else:
        return sum(model.x[i,j] for j in model.J if i!=j) <= 1

def origin_stage1(model, i):
    return sum(model.x[i,j] for j in model.stage1 if i!=j) == 1

def origin_stage2_3(model, i):
    return sum(model.x[i,j] for j in model.stage2stage3 if i!=j) == 0

def stage1_stage3(model, i):
    return sum(model.x[i,j] for j in model.stage3 if i!=j) == 0

def stage1_stage2(model):
    return sum(model.x[i,j] for i in model.stage1 for j in model.stage2) == 1

def stage2_stage3(model):
    return sum(model.x[i,j] for i in model.stage2 for j in model.stage3) == 1

# we have to avoid cycles
model.cycles = Constraint(model.I, model.J, rule=cycles) 

# there are not arrivals to origin city(1) and we can arrive just once to each of the othre cities
model.arrivals = Constraint(model.J, rule=arrivals) 

# from stage 3 cities (8, 9) there is only one departure since one of them is the destiny city. Form the other cities there is only one departure
model.departures = Constraint(model.I, rule=departures)

# from origin city there are only flights to 1st stage cities (2, 3, 4)
model.origin_stage1 = Constraint(model.origin, rule=origin_stage1)
model.origin_stage2_3 = Constraint(model.origin, rule=origin_stage2_3)

# from 1st stage cities (2, 3, 4) there are not flights to 3rd stage cities (8, 9)
model.stage1_stage3 = Constraint(model.stage1, rule=stage1_stage3)

# from 1st stage there is only one flight to 2nd stage cities (5, 6, 7)
model.stage1_stage2 = Constraint(rule=stage1_stage2)

# from 2nd stage (5, 6, 7) there is only one flight to 3rd stage cities (8, 9)
model.stage2_stage3 = Constraint(rule=stage2_stage3)



In [ ]:
!pyomo solve  european_trip.py  european_trip.dat --solver=glpk --summary 


In [ ]:
!type results.yml

### ANALYSIS OF THE RESULTS

After performing teh optimization problem we have derive that the most optimal path is:

**Madrid -Milan - Paris - Luxemburgo - Berlin - Munich - Viena - Budapest - Split** and it will cost us 555 €
<img src="img\resultado.png">


# NON-LINEAR OPTIMIZATION PROBLEM

## THE FACTORY POLLUTION

### PROBLEM STATMENT
We are studying the pollution and enviromental impact of an enterprise of prepared food. We know that this enterprise follows a process composed of 4 phases:
- In the first phase, the products are weighted and two different machines are used (1, 2). The emissions of this phase are 30 Liters of CO2 multiplied by the square of the minutes the machines are working.
- In the second phase, the products are cutted and prepared for the cooking part. For this phase, we have avaliable 4 machines (3, 4, 5, 6). The emissions for this phase are the same for 3 of the machines, but one of it pollutes the square of the rest. Therefore, the emissions are 20 L of CO2 multiplied for the cube of all the minutes the machines are working (taking into account that machine 4 of it pollutes more).
- In the last part, the food is cooked and packed. For this phase we have 2 machines (7, 8), the pollution of this machines is given by the product of the minutes both are working times 50 Liters per minute, considring that machine 7 pollutes more.

We are interested in minimize the pollutions of this enterprise, but having in mind that each process needs a minimum of time.
- The first phase needs at least 3.000 minutes to be accomplished well.
- The second phase needs to be in production for at least 8.000 minutes.
- The third phase must be working for t least 1.500 minutes.

Moreover, we must consier that we are expecting for a benefit of at least 20.000 €
- The first phase generates 40 € for each minute working.
- The second phase generates 60 € for each minute working.
- The third phase generates 30 € for each minute working.


### PROBLEM FORMULATION
#### Variables:
$x_{i}$ = minutes the machine $i$ is working 

#### Equations:
- $Pollution(X) = 30(x_{1} +x_{2})^2 + 20(x_{3} + x_{4}^2 + x_{5} + x_{6})^3 + (50 x_{7}^5 x_{8}) $
- $ Benefit(X) = 40(x_{1}+x_{2}) + 60(x_{3} + x_{4} + x_{5} + x_{6}) + 30(x_{7} + x_{8}) $

#### Objective:
Minimize the pollution of all processes:             $\min_{x} Pollution(X)$

#### Constraints
- First phase needs at least 3.000 mins: $(x_{1} +x_{2}) \geq 300$
- Second phase needs at least 8.000 mins: $(x_{3} + x_{4} + x_{5} + x_{6}) \geq 800$
- Third phase needs at least 1.500 mins: $(x_{7} + x_{8}) \geq 150$
- The minimum benefit achieved is 20.000€: $Benefit(X) \geq 2000$


### CODING THE PROBLEM

In [ ]:
from pyomo.environ import *

#declare the model
model = ConcreteModel()

# declare the variables
model.x1 = Var(within = NonNegativeReals, initialize=0.0)
model.x2 = Var(within = NonNegativeReals, initialize=0.0)
model.x3 = Var(within = NonNegativeReals, initialize=0.0)
model.x4 = Var(within = NonNegativeReals, initialize=0.0)
model.x5 = Var(within = NonNegativeReals, initialize=0.0)
model.x6 = Var(within = NonNegativeReals, initialize=0.0)
model.x7 = Var(within = NonNegativeReals, initialize=0.0)
model.x8 = Var(within = NonNegativeReals, initialize=0.0)

# define the objective function
def pollution(model):
    return (30.0*(model.x1 + model.x2)**2 + 20.0*(model.x3 + model.x4**2 + model.x5 + model.x6)**3 + 50.0*model.x7**5*model.x8)
model.objective = Objective(rule=pollution, sense=minimize) 
 
# define the constraints
def benefit(model):
    return (40.0*(model.x1 + model.x2) + 60.0*(model.x3 + model.x4 + model.x5 + model.x6) + 30.0*(model.x7 + model.x8))

model.constraint1 = Constraint(expr=benefit(model)>=2000)
model.constraint2 = Constraint(expr=(model.x1 + model.x2)>=300)
model.constraint3 = Constraint(expr=(model.x3 + model.x4 + model.x5 + model.x6)>=800)
model.constraint4 = Constraint(expr=(model.x7+model.x8)>=150)

# obtain the solution
solver = SolverFactory("ipopt") # define the solver
#model.dual = Suffix(direction=Suffix.IMPORT)
solution = solver.solve(model) # solve

# Display solution
print('x1=', model.x1())
print('x2=', model.x2())
print('x3=', model.x3())
print('x4=', model.x4())
print('x5=', model.x5())
print('x6=', model.x6())
print('x7=', model.x7())
print('x8=', model.x8())

display(model.objective)
display(model.constraint1)

# Display lagrange multipliers
#display(model.dual)


### ANALYSIS AND RESULTS:
After having used the solver **ipopt** solver, we obtain that we will use:
- machine one and 2 for 150 minutes aprox each of them.
- machine 3, 5 and 6 for 267 minutes aprox and machine 4 just for 30 seconds since it pollutes much more (the square of them)
- machine 8 for 9695 minutes aprox and machine 7 almost nothing since it pollites much more (to the power of 5).

Using machines in with distribution we obtain that we will have the minimum number of emissions at  10.233.102.692 L of CO2 and a benefit of 350.851 €
